In [1]:
import requests
import json
import pandas as pd

In [2]:

api_key = "ef4c5176645445238294a9fbf5fa8ad1"
subscription_name = "comp90024"
url = "https://gateway.api.epa.vic.gov.au/environmentMonitoring/v1/sites/parameters?environmentalSegment=air"

headers = {
    "User-Agent": 'curl/8.4.0',
    "Cache-Contrl": "no-cache",
    'X-API-Key': api_key
}


response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
        
else:
    print("query fail:", response.status_code)

print(response.text)

{
  "totalRecords" : 93,
  "records" : [
    {
      "siteID" : "77062cb7-3e3b-4984-b6d0-03dda76177f2",
      "siteName" : "Box Hill",
      "siteType" : "Standard",
      "geometry" : {
        "type" : "Point",
        "coordinates" : [
          -37.8287277,
          145.1324
        ]
      },
      "siteHealthAdvices" : [
        {
          "since" : "2024-05-09T05:00:00Z",
          "until" : "2024-05-09T06:00:00Z",
          "healthParameter" : "PM2.5",
          "averageValue" : 30.25,
          "unit" : "&micro;g/m&sup3;",
          "healthAdvice" : "Fair",
          "healthAdviceColor" : "#EEC900",
          "healthCode" : "1022"
        }
      ],
      "parameters" : [
        {
          "name" : "PM2.5",
          "timeSeriesReadings" : [
            {
              "timeSeriesName" : "1HR_AV",
              "readings" : [
                {
                  "since" : "2024-05-09T05:00:00Z",
                  "until" : "2024-05-09T06:00:00Z",
                  "averageV

In [3]:
file_path = "data.json"

# 将数据写入 JSON 文件
with open(file_path, "w") as json_file:
    json.dump(response.json(), json_file)


In [4]:

def json_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    # 将JSON数组转换为DataFrame
    df = pd.json_normalize(data['records'])

    # 显示DataFrame
    print(df.head())
    return df



In [5]:

df= json_to_dataframe(file_path)

# 显示 DataFrame 的前几行数据
print(df.head())

                                 siteID         siteName  siteType  \
0  77062cb7-3e3b-4984-b6d0-03dda76177f2         Box Hill  Standard   
1  c69ed768-34d2-4d72-86f3-088c250758a8       Alphington  Standard   
2  ee780b50-0240-4c7e-99f8-0df759caf3a3        Churchill  Standard   
3  cddf953a-b932-4918-97ea-1d19583d507a  Traralgon South    Sensor   
4  70584bae-a7e7-4ae9-adf5-1e8e92b15386        Traralgon  Standard   

                                   siteHealthAdvices  \
0  [{'since': '2024-05-09T05:00:00Z', 'until': '2...   
1  [{'since': '2024-05-09T05:00:00Z', 'until': '2...   
2  [{'since': '2024-05-09T05:00:00Z', 'until': '2...   
3  [{'since': '2024-05-09T05:00:00Z', 'until': '2...   
4  [{'since': '2024-05-09T05:00:00Z', 'until': '2...   

                                          parameters geometry.type  \
0  [{'name': 'PM2.5', 'timeSeriesReadings': [{'ti...         Point   
1  [{'name': 'SO2', 'timeSeriesReadings': [{'time...         Point   
2  [{'name': 'PM10', 'timeSeries

## preview数据集

In [6]:
df.columns

Index(['siteID', 'siteName', 'siteType', 'siteHealthAdvices', 'parameters',
       'geometry.type', 'geometry.coordinates', 'cameraImageURL',
       'timeLapseURL'],
      dtype='object')

In [7]:
df['siteName'].unique()


array(['Box Hill', 'Alphington', 'Churchill', 'Traralgon South',
       'Traralgon', 'Tyers North', 'Dandenong', 'Moe', 'Boolarra South',
       'Traralgon East', 'Boolarra', 'Yinnar', 'Morwell East',
       'Geelong South', 'Willow Grove', 'Yallourn North', 'Point Cook',
       'Flynns Creek', 'Melbourne CBD', 'Tyers', 'Callignee',
       'Hernes Oak', 'Wangaratta', 'Morwell South', 'Brighton', 'Echuca',
       'Healesville', 'Warrnambool', 'Mildura', 'Horsham', 'Altona North',
       'Ballarat', 'Bairnsdale', 'Castlemaine', 'Beechworth', 'Benalla',
       'Ararat', 'Cobden', 'Camperdown', 'Colac', 'Maffra', 'Gisborne',
       'Macedon', 'Lancefield', 'Kyneton', 'Heathcote', 'Kinglake',
       'Daylesford', 'Heywood', 'Hamilton', 'Kerang', 'Lorne',
       'Mallacoota', 'Shepparton', 'Orbost', 'Sale', 'Myrtleford',
       'Mansfield', 'Portland', 'Ouyen', 'Torquay', 'Wonthaggi',
       'Warragul', 'Yarrawonga', 'Wodonga', 'Broadford', 'Yarra Glen',
       'Sunbury', 'Lakes Entrance', '

## 预处理

In [13]:
column_0_data = df.iloc[:, 0]
column_1_data = df.iloc[:, 1]
column_4_data = df.iloc[:, 4]
df_main_v2 = pd.DataFrame()
for i, item in enumerate(column_4_data):
    if isinstance(item, float):
        
        continue  # 跳过 float 类型的数据
    
    if isinstance(item, list) :
        for type in item: #循环每种监控器类型读到的数据
            if type["name"] ==  'PM2.5':
                for tsr in type['timeSeriesReadings']: # 读取每种时间段的数据
                    tsr_name = tsr['timeSeriesName']
                    # 遍历 readings
                    for reading in tsr['readings']:
                        # 将每个 reading 转换为 DataFrame
                        df_temp = pd.DataFrame([reading])
                        # 添加 name 和 timeSeriesName 列
                        df_temp['name'] = 'PM2.5'
                        df_temp['timeSeriesName'] = tsr_name
                        df_temp['sit_id'] = column_0_data[i]
                        df_temp['sit_name'] = column_1_data[i]
                        # 将结果添加到主 DataFrame
                        df_main_v2 = pd.concat([df_main_v2, df_temp])

# 重新排序列
df_main_v2 = df_main_v2[['sit_id','sit_name', 'name', 'timeSeriesName', 'since', 'until', 'averageValue', 'unit', 'confidence', 'totalSample', 'healthAdvice', 'healthAdviceColor', 'healthCode']]

# 显示 DataFrame


                    
        

sit_id               77062cb7-3e3b-4984-b6d0-03dda76177f2
sit_name                                         Box Hill
name                                                PM2.5
timeSeriesName                                     1HR_AV
since                                2024-05-09T05:00:00Z
until                                2024-05-09T06:00:00Z
averageValue                                        30.25
unit                                     &micro;g/m&sup3;
confidence                                         108.33
totalSample                                            13
healthAdvice                                         Fair
healthAdviceColor                                 #EEC900
healthCode                                           1022
Name: 0, dtype: object


In [17]:
print(df_main_v2.iloc[4])

sit_id               ee780b50-0240-4c7e-99f8-0df759caf3a3
sit_name                                        Churchill
name                                                PM2.5
timeSeriesName                                     1HR_AV
since                                2024-05-09T05:00:00Z
until                                2024-05-09T06:00:00Z
averageValue                                         1.13
unit                                     &micro;g/m&sup3;
confidence                                         108.33
totalSample                                            13
healthAdvice                                         Good
healthAdviceColor                                 #42A93C
healthCode                                           1021
Name: 0, dtype: object


In [14]:
len(df_main_v2)

56

In [15]:
df_main_v2.to_csv("air_data.csv", index=False)

In [18]:
df_3 = pd.read_csv("air_data.csv")

In [22]:
filtered_rows = df_3[df_3['timeSeriesName'] == '24HR_AV']

In [25]:
filtered_rows.to_csv("air_24h+df.csv", index=False)